# Fetching data from [cryptocompare.com](https://www.cryptocompare.com)

In [1]:
import requests

class CryptoApi:
    def __init__(self) -> None:
        self.mother_endpoint = "https://min-api.cryptocompare.com/data"

        with open('token.txt', 'r') as f:
            self.TOKEN = f.readline()

        self.HEADER = {'Authorization': self.TOKEN,
                       "Content-Type": 'application/json'}

        endpoint = f'{self.mother_endpoint}/v2/histo{period}?fsym={crypto}&tsym={currency}&limit={period_count}'
        response = requests.get(endpoint, params={'allData':allData}, headers=self.HEADER)
        
        return response.json()
    
    def execute_custom_getrequest(self, endpoint:str):
        return requests.get(endpoint, headers=self.HEADER)
    
data = CryptoApi().get_data('btc','usd','day',29)

In [28]:
data

{'Response': 'Success',
 'Message': '',
 'HasWarning': False,
 'Type': 100,
 'RateLimit': {},
 'Data': {'Aggregated': False,
  'TimeFrom': 1679961600,
  'TimeTo': 1680048000,
  'Data': [{'time': 1679961600,
    'high': 27501.44,
    'low': 26660.19,
    'open': 27145.09,
    'volumefrom': 38686.66,
    'volumeto': 1046551233.93,
    'close': 27274.9,
    'conversionType': 'direct',
    'conversionSymbol': ''},
   {'time': 1680048000,
    'high': 28640.99,
    'low': 27261.75,
    'open': 27274.9,
    'volumefrom': 29355.13,
    'volumeto': 822534043.78,
    'close': 28368.82,
    'conversionType': 'direct',
    'conversionSymbol': ''}]}}

In [32]:
mystr = '''I plan to use Twitter data for my pet-project and answer the question whether Elon Musk's tweets have an effect on cryptocurrencies, this is some kind of a research as well
not for commercial use, '''

len(mystr)

197

In [21]:
'''from datetime import datetime

    def get_data(self, crypto:str, currency:str, period:str, period_count:int, allData=0):
        '''Returns crypto summary for a given period in specified currency.

        Args:
            crypto: str
                BTC/ETH/DOGE etc.
            currency: str
                USD/EUR/UAH etc.
            period: str 
                day/hour/minute.
            period_count: int
                last n of a period (n days)
                n = 1 returns revious day/hour/minute + current
            allData: int
                bool doen't work
                1 - get all records;
                0 - get specified amount of period_count.
        Returns:
            dict: json containing request's response.
        '''        

        endpoint = f'{self.mother_endpoint}/v2/histo{period}?fsym={crypto}&tsym={currency}&limit={period_count}'
        response = requests.get(endpoint, params={'allData':allData}, headers=self.HEADER)
        
        return response.json()
    
    def execute_custom_getrequest(self, endpoint:str):
        return requests.get(endpoint, headers=self.HEADER)
    
data = CryptoApi().get_data('btc','usd','day',29)

'from datetime import datetime\n\ndatetime.fromtimestamp(1695054322)'

In [2]:
'''from datetime import datetime

datetime.fromtimestamp(1695054322)'''

'from datetime import datetime\n\ndatetime.fromtimestamp(1695054322)'